In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Keys, ActionChains
import time, os

profile = 'Profile 2'#Vó
profile = 'Profile 7'#Mineiro
profile = 'Default'#Jaque

os.system('taskkill /F /IM msedge.exe')

options = webdriver.EdgeOptions()
#options.add_argument('user-data-dir=C:\\Users\\vinic\\AppData\\Local\\Google\\Chrome\\User Data')
options.add_argument('user-data-dir=C:\\Users\\vinic\\AppData\\Local\\Microsoft\\Edge\\User Data')
options.add_argument(f'profile-directory={profile}')
##options.add_argument('headless')

driver = webdriver.Edge(options=options)




In [16]:
'Data:\n22/10/2024 - 23:33'[6:16]

'22/10/2024'

In [ ]:
import datetime
from navegador import fechar_tudo_zord, get_element_by_text, click

continuar = True
contas = ['BAKESHOP23', 'INKALION']

while continuar:
    try:
        
        driver.get('https://app.mercadoturbo.com.br/sistema/venda/vendas_ml')
        time.sleep(2)

        vendas_list = []
        for conta in contas:
            driver.find_element(By.CSS_SELECTOR, 'li form div.ui-selectonemenu-trigger > span').click()
            time.sleep(1)
            get_element_by_text(driver, conta, texto_exato=False, tipo_tag='li').click()
            time.sleep(8)

            for venda in driver.find_elements(By.CSS_SELECTOR, 'td.HeaderTextAlignLeft.CursorDefault.ColunaItemsVendas'):
                num_venda = venda.find_element(By.CSS_SELECTOR, 'div.p-d-flex.p-ai-center > a > span:nth-child(2)').text
                qtdd = venda.find_element(By.CSS_SELECTOR, 'div.Fs20').text.replace('x', '')
                sku = venda.find_element(By.CSS_SELECTOR, 'span.p-ml-1').text
                vendas_list.append({'num_venda' : num_venda ,'sku' : sku, 'qtdd' : qtdd, 'status' : '1' if len(venda.find_elements(By.CSS_SELECTOR, 'div.Red')) == 0 else '2', 'conta' : conta})


        vendas_baixar_list = []
        with open('controle_estoque.txt', 'r+') as file:
            file_content = file.read()
            for venda in vendas_list:
                compare = '|'.join([venda['num_venda'], venda['sku'], venda['qtdd'], venda['status'], venda['conta']])
                print(compare)
                if file_content.count(compare) == 0:
                    vendas_baixar_list.append(venda)

            if len(vendas_baixar_list) > 0:
                driver.get('https://taticalmilitaria.painel.magazord.com.br/')
                fechar_tudo_zord(driver, 1)
                driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[7]/div').click()
                driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[7]/ul/li[1]/div/span').click()
                
                time.sleep(2)
                get_element_by_text(driver, 'Movimentações').click()
                get_element_by_text(driver, 'Movimentar').click()

                time.sleep(2)
                for venda in vendas_baixar_list:
                    driver.find_element(By.CSS_SELECTOR, '[name="deposito"]').send_keys('Padrão Tatical Militaria')
                    driver.find_element(By.CSS_SELECTOR, '[name="deposito"]').send_keys(Keys.TAB)
                    time.sleep(1)

                    driver.find_elements(By.CSS_SELECTOR, '[name="produtoDerivacao/codigo"]')[1].send_keys(venda['sku'])
                    driver.find_element(By.CSS_SELECTOR, '[name="produtoDerivacao/codigo"]').send_keys(Keys.TAB)
                    time.sleep(1)
                    
                    obsAdd = '. Conta:' + venda['conta']
                    if venda['status'] == '1':
                        driver.find_element(By.CSS_SELECTOR, '[name="tipoOperacao"]').send_keys('Saída')
                        driver.find_element(By.CSS_SELECTOR, '[name="tipoOperacao"]').send_keys(Keys.TAB)
                    else:
                        driver.find_element(By.CSS_SELECTOR, '[name="tipoOperacao"]').send_keys('Entrada')
                        driver.find_element(By.CSS_SELECTOR, '[name="tipoOperacao"]').send_keys(Keys.TAB)
                        obsAdd = ' Pagamento Cancelado'
                    time.sleep(1)

                    driver.find_element(By.CSS_SELECTOR, '[name="tipo"]').send_keys('Físico')
                    driver.find_element(By.CSS_SELECTOR, '[name="tipo"]').send_keys(Keys.TAB)
                    time.sleep(1)

                    driver.find_element(By.CSS_SELECTOR, '[name="observacao"]').send_keys('Venda ML Referência: ' + venda['num_venda'] + obsAdd)
                    driver.find_element(By.CSS_SELECTOR, 'td.x-form-item-body[role="presentation"] > input[data-errorqtip]:nth-child(2)').send_keys(venda['qtdd'])
                    time.sleep(1)

                    driver.find_element(By.CSS_SELECTOR, '[name="origemMovimento"]').send_keys('8')
                    driver.find_element(By.CSS_SELECTOR, '[name="origemMovimento"]').send_keys(Keys.TAB)
                    time.sleep(1)

                    driver.find_element(By.CSS_SELECTOR, '[name="dataHora"]').send_keys(datetime.datetime.now().strftime("%d%m%Y%H%M%S"))
                    time.sleep(1)
                    
                    click(driver, get_element_by_text(driver, 'Gravar'))
                    time.sleep(3)

                    file.write(';')
                    file.write('|'.join([venda['num_venda'], venda['sku'], venda['status'], venda['qtdd'], venda['conta']]))
        time.sleep(60)
    except:
        pass

In [ ]:
from selenium.webdriver.common.keys import Keys
import datetime, re

'''
por_pagina = driver.find_element(By.CSS_SELECTOR, '[name="perpage"]')
por_pagina.click()

filtro_situacao = driver.find_element(By.CSS_SELECTOR, '[name="pedidoSituacao/codigo"]')
filtro_situacao.click()
time.sleep(0.5)
driver.find_element(By.XPATH, "//li[text()='4 - Aprovado']").click()
time.sleep(0.5)
driver.find_element(By.XPATH, "//li[text()='5 - Aprovado e Integrado']").click()
time.sleep(0.5)
filtro_situacao.send_keys(Keys.ENTER)

atual = datetime.datetime.today()
d1  = atual - datetime.timedelta(days = 1)
d30 = atual - datetime.timedelta(days = 30)


sData1  = d1.strftime("%d/%m/%Y")
sData30 = d30.strftime("%d/%m/%Y")


filtro_data = driver.find_element(By.CSS_SELECTOR, f'td > input[value="{sData1}"]')
filtro_data.click()
filtro_data.send_keys(sData30)


driver.find_element(By.XPATH, "//*[text()='Atualizar']").click()
'''


for tr in driver.find_elements(By.CSS_SELECTOR, 'tr.x-grid-row'):
    num_pedido = tr.find_element(By.CSS_SELECTOR, 'td:nth-child(3)').text
    print(num_pedido)

    
    tr.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').click()
    driver.find_element(By.XPATH, "//*[text()='Visualizar']").click()
    time.sleep(2)

    nome = driver.find_elements(By.XPATH, "//*[text()='Cliente']//..//..//..")[-1].text

    text_pagina = driver.find_element(By.CSS_SELECTOR, 'body').text
    re_cpf_cnpj = re.search(r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b', text_pagina)
    if not re_cpf_cnpj:
        re_cpf_cnpj = re.search(r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b', text_pagina)
    cpf_cnpj = re_cpf_cnpj.group()
    dict_venda = dict(nome=nome,documento=cpf_cnpj,num_pedido=num_pedido)
    print(dict_venda)

    for produto_html in driver.find_elements(By.CSS_SELECTOR, '.swiper-slide-active'):
        nome_produto = produto_html.find_element(By.CSS_SELECTOR, '.nomeProduto').text    
        print(nome_produto)


    break






In [ ]:
import pandas as pd
import re


cores = \
['PRETO', 'AZUL MARINHO', 'AZUL HORIZONTE', 'AZUL', 'MARROM', 'VERDE OLIVA', 'VERDE', 'WARSKIN BLACK', 'WARSKIN B.', 'WARSKIN', 'CÁQUI', 'CAQUI', 'TAN', 'DESERT' , 'COYOTE', \
'CINZA', 'GRAFITE', 'CAMURÇA', 'CHUMBO', 'WOODLAND', 'DIGITAL ACU', 'DIGITAL DESERT', 'CAQUI SAND PRF', 'CAQUI MOJAVE'
]

tamanhos = [ \
    '36', '37','38','39','40','41','42','43','44','45','46','47','48','49','50', \
    'P', 'M', 'GG', 'G', 'XGG', 'EG'

]

df_catalogo = pd.read_excel('catalogo_invictus.xlsx')
'''
#driver.get('https://taticalmilitaria.painel.magazord.com.br/')
#time.sleep(5)
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/div/span').click()
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/ul/li[2]/div').click()
driver.find_element(By.XPATH, '//*[@id="area-superior"]/div/div[1]/ul/li[1]/ul/li[2]/ul/li[1]/div').click()
por_pagina = driver.find_element(By.CSS_SELECTOR, '[name="perpage"]')

por_pagina.send_keys('500')
'''

for i, row in df_catalogo.iterrows():
    ini = 600
    if i < ini:
        continue

    if i >= ini+1000:
        break

    print(row['Hierarquia Produto - nome_produto'] + ' - ' + str(row['código EAN']) + ' - Linha:' + str(i))
    ean = str(row['código EAN'])
    ncm = str(row['código NCM']).replace('.', '')
    descricao_produto_generico = row['Hierarquia Produto - nome_produto']
    descricao_produto_completa = row['Hierarquia Produto - nome_produto'].upper()\
        .replace('PRETA', 'PRETO')\
        .replace(' TAM. ', ' ')\
        .replace(' TAM ', ' ')\
        .replace(' TAM Nº ', ' ')\
        .replace(' Nº ', ' ')\
        .replace(' DE ', ' ')\
        .replace(' INVICTUS', '')\
        .replace(' TÁTICO', '')\
        .replace(' TATICO', '')\
        .replace(' TÁTICA', '')\
        .replace(' TATICA', '')


    cor = ''
    for i_cor in cores:
        if descricao_produto_completa.rfind(i_cor) > 0:
            cor = i_cor
            break

    tam = ''
    for i_tam in tamanhos:
        if descricao_produto_completa.find(cor + ' ' + i_tam) > 0:
            tam = i_tam
            break

    filtro_descricao = driver.find_element(By.CSS_SELECTOR, '[name="descricao_completa"]')
    filtro_descricao.clear()
    filtro_descricao.send_keys(descricao_produto_completa)

    driver.find_element(By.XPATH, "//*[text()='Atualizar']").click()
    time.sleep(1.2)

    #Se não retornou resultado nenhum    
    if len(driver.find_elements(By.CSS_SELECTOR, '.x-grid-row-checker')) == 0:
        continue

    checkbox = driver.find_element(By.CSS_SELECTOR, '.x-grid-row-checker')
    checkbox.click()
    driver.find_element(By.XPATH, "//*[text()='Alterar']").click()
    driver.find_element(By.XPATH, "//*[text()='Alterar Produto (Novo)']").click()
    time.sleep(3)
    
    alterou = False
    controle_paginacao = True
    achou_variacao = False
    while controle_paginacao == True:
        for el in driver.find_elements(By.CSS_SELECTOR, '.sc-fTFjTM.frSKuo'):
            variacao_input = el.find_elements(By.CSS_SELECTOR, '[name="nome"]')[1].get_attribute('value').upper()

            variacao_planilha = ''
            if len(cor) > 0 and len(tam) > 0:
                variacao_planilha = cor + ' - ' + tam
            elif len(cor) > 0:
                variacao_planilha = cor
            elif len(tam) > 0:
                variacao_planilha = tam

            variacao_input    = variacao_input.replace('VERDE OLIVA', 'VERDE')\
                                                .replace('AZUL MARINHO', 'AZUL')\
                                                    .replace('VERDE MILITAR', 'VERDE')
            variacao_planilha = variacao_planilha.replace('VERDE OLIVA', 'VERDE')\
                                                .replace('AZUL MARINHO', 'AZUL')\
                                                    .replace('VERDE MILITAR', 'VERDE')
            print('Variacao planilha: ', variacao_planilha, ' - Variacao input:', variacao_input)
            if variacao_input == '' or variacao_input == variacao_planilha:
                achou_variacao = True                
                
                el_ean = el.find_element(By.CSS_SELECTOR, '[name="ean"]')
                print('Variação achada: ', variacao_planilha, ' ', 'Preencher...' if el_ean.get_attribute('value') == '' else 'Já preenchido')
                if el_ean.get_attribute('value') == '':
                    el_ean.send_keys(ean)
                    alterou = True
                controle_paginacao = False
                break
        if len(driver.find_elements(By.XPATH, "//*[text()='Próximo']")) > 0:
            if driver.find_element(By.XPATH, "//*[text()='Próximo']").get_attribute('disabled'):
                controle_paginacao = False
            else:
                driver.find_element(By.XPATH, "//*[text()='Próximo']").click()
                time.sleep(0.5)

        else:
            controle_paginacao = False

    time.sleep(0.3)
    driver.find_element(By.XPATH, "//*[text()='Tributação e Dimensão']").click()
    time.sleep(0.7)
    
    el_ncm = driver.find_element(By.CSS_SELECTOR, "[name='ncm']")
    if el_ncm.get_attribute('value') == '':
        el_ncm.send_keys(ncm)
        alterou = True

    alterou_dimensao = False    
    el_peso = driver.find_element(By.CSS_SELECTOR, "[name='peso-0']")
    if el_peso.get_attribute('value') == '':
        el_peso.send_keys('0,500')
        alterou_dimensao = True    
    
    el_altura = driver.find_element(By.CSS_SELECTOR, "[name='altura-0']")
    if el_altura.get_attribute('value') == '':
        el_altura.send_keys('7,00')
    
    el_largura = driver.find_element(By.CSS_SELECTOR, "[name='largura-0']")
    if el_largura.get_attribute('value') == '':
        el_largura.send_keys('20,00')
    
    el_comprimento = driver.find_element(By.CSS_SELECTOR, "[name='comprimento-0']")
    if el_comprimento.get_attribute('value') == '':
        el_comprimento.send_keys('30,00')

    if alterou:
        driver.find_element(By.XPATH, "//*[text()='Salvar']").click()
        time.sleep(1.5)
    
    driver.find_element(By.XPATH, "//*[text()='Fechar']").click()
    time.sleep(0.2)

    deu_erro = False
    if len(driver.find_elements(By.XPATH, "//*[text()='Tributação e Dimensão']")):
        deu_erro = True
        alterou = False
    
    if deu_erro:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - ERRO\n')
    elif alterou_dimensao:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - Dimensão alterada\n')
    elif not achou_variacao:
        with open('erros.txt', 'a') as file:
            file.write(descricao_produto_completa + ' - Variação não encontrada\n')




    checkbox.click()

In [ ]:
import os, time

os.system("taskkill /im msedge.exe /f")

options = webdriver.EdgeOptions()
usu = 'Default'
caminho = 'C:\\Users\\vinic\\AppData\\Local\\Microsoft\\Edge\\User Data'
options.add_argument(f'user-data-dir={caminho}')
options.add_argument(f'profile-directory={usu}')

driverEdge = webdriver.Edge(options=options)

time.sleep(5)

In [ ]:
"""
import sqlite3
from pathlib import Path


ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()
"""

driver.get(f'https://www.mercadolivre.com.br/vendas/omni/lista?startPeriod=WITH_DATE_CLOSED_6M_OLD&sort=DATE_CLOSED_DESC')
time.sleep(10)
vendas_lista_html = \
    driver.find_elements(By.CSS_SELECTOR, '#desktop > div.sc-page > div > div.sc-tab-content > div > div.sc-list.sc-list-marketplace > div.andes-card')

for venda_html in vendas_lista_html:
    numero_venda = venda_html.find_element(By.CSS_SELECTOR, 'div > div.identification-row > div.left-column > div.left-column__pack-id').text
    print(numero_venda)

"""        
connection.commit()
cursor.close()
connection.close()
"""


Funções de envio de mensagem e shit_enter(para quebrar linha dentro do input de msg)

In [ ]:
import requests

url = f"http://177.202.248.188:8081/message/sendText/tatical"
numero = '5547996329923'
numero = '5577999956924'
msg = '''
Hope!🫡

*Luciano Garcia?*
Aqui é da Tatical Militaria, tudo bem guerreiro(a)?
Recebi seu pedido via mercado livre:*1x Boné Legionário Responder 2.0 - Galapagos - VERDE*
Confere?

Caso você não seja a pessoa indicada acima desculpe o incômodo e desconsidere essa mensagem
🙏🏽

'''
payload = {
    "number": numero,
    "text": msg,
    "delay": 123
}
headers = {
    "apikey": '4?2S45_}?pjxeI#Z;5',
    "Content-Type": "application/json"
}
requests.HTTPError

response = requests.post(url, json=payload, headers=headers, timeout=4)
response.raise_for_status()

ConnectTimeout: HTTPConnectionPool(host='177.202.248.188', port=8081): Max retries exceeded with url: /message/sendText/tatical (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001541A21F790>, 'Connection to 177.202.248.188 timed out. (connect timeout=4)'))

In [ ]:
import datetime

# Data e hora atual
atual = datetime.datetime.today()
d = datetime.timedelta(days = 30)
atual = atual - d
s1 = atual.strftime("%d/%m/%Y")

s1